# 0. Imports aand Arguments


In [1]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

from scripts.evaluation_utils import delayed_impact_csv, immediate_impact_csv, delayed_impact_2csv

In [2]:
data_path = '../results/german_all/'
folders= ['dt','gnb','lgr','gbt']
folders= ['dt','gnb','gbt']

# 1. Build useful CSVs

## 1.2 FP/TP/TN/FN Ratios

In [3]:
# Ratios
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts(normalize=True)
    df = df.reset_index()
    df = df.rename(columns= {0:'Ratio'})
    
    df = df.pivot(index='Category', columns='ID')['Ratio']
    df = df.fillna(0)
    df = df.round(decimals = 3)
    df.to_csv(f'{data_path}{f}/{f}_type_ratios.csv')

In [4]:
# Absolute numbers
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts()
    df = df.reset_index()
    df = df.rename(columns= {0:'Number'})
    
    df = df.pivot(index='Category', columns='ID')['Number']
    df = df.fillna(0)
    df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

## 1.1 Impact

In [5]:
delayed_impact_csv(data_path,'Black', folders)
delayed_impact_csv(data_path,'White', folders)

Group:  Black 
 DataFrame: 
                 DT     GNB     GBT
Constraint                        
Unmitigated  92.42  105.08  108.97
DP           99.81  110.69  113.57
EO           95.82  112.10  106.48
EOO          98.49  112.19  103.64
FPER         98.12  102.01  108.76
ERP          99.19  110.17  109.85
Group:  White 
 DataFrame: 
                  DT     GNB     GBT
Constraint                         
Unmitigated   99.15  117.17  123.27
DP           100.17  115.65  120.95
EO            98.08  112.23  121.64
EOO           97.86  115.57  122.15
FPER          98.57  116.50  121.27
ERP           99.03  107.17  121.02


C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


In [6]:
immediate_impact_csv(data_path,'Black', folders)
immediate_impact_csv(data_path,'White', folders)

Group:  Black 
 DataFrame: 
                 DT    GNB    GBT
Constraint                      
Unmitigated  0.375  0.511  0.466
DP           0.466  0.511  0.489
EO           0.375  0.534  0.466
EOO          0.375  0.534  0.466
FPER         0.352  0.511  0.466
ERP          0.375  0.534  0.466
Group:  White 
 DataFrame: 
                 DT    GNB    GBT
Constraint                      
Unmitigated  0.462  0.566  0.585
DP           0.434  0.547  0.566
EO           0.434  0.557  0.585
EOO          0.434  0.557  0.585
FPER         0.462  0.538  0.585
ERP          0.424  0.462  0.566


C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


In [7]:
delayed_impact_2csv(data_path,'Black', folders)
delayed_impact_2csv(data_path,'White', folders)

Group:  Black 
 DataFrame: 
                  DT     GNB    GBT
Constraint                        
Unmitigated  17.025  18.825  15.30
DP           18.750  15.375  12.75
EO           15.375  16.200  15.30
EOO          13.725  16.200  15.30
FPER         12.825  18.825  13.65
ERP          13.725  19.650  13.65
Group:  White 
 DataFrame: 
                  DT     GNB     GBT
Constraint                         
Unmitigated  20.475  16.575  14.550
DP           18.825  16.575  15.225
EO           18.825  19.050  15.150
EOO          19.425  16.950  15.150
FPER         20.475  15.525  15.150
ERP          17.700  15.525  13.875


C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


# 2. Analyzing Scores

## 2.1 Extractig Scores from csv into dataframes

In [ ]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}

for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.round(0)

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white

## 2.2 Checking for normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [ ]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm")
        if p > 0.01:
            print(col,',p:',p)
    print('Check for norm Distributions done')

## 2.3 Significance of Score Distributions with Mann Whitney U test:


In [ ]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

### Variance of Distributions Each Model against each other
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [ ]:
def p_values_mwu_bymodel(dfs,model=''):
    p_vals = pd.DataFrame(data={'Classifier': []})
    p_signi = pd.DataFrame(data={'Classifier': []})
    
    for c1,df1 in dfs.items():
        col_signi = []
        col_vals = []
        idx = []
        
        data1 = df1[model].dropna(axis=0)
        
        for c2,df2 in dfs.items():
            idx.append(c2)
            
            data2 =df2[model].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data1, data2)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c1] = col_signi
        p_vals[c1] = col_vals
        
    p_vals['Classifier'] = idx
    p_vals = p_vals.set_index('Classifier')
    p_signi['Classifier'] = idx
    p_signi = p_signi.set_index('Classifier')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_{model}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_{model}.csv')

In [ ]:
for col in classifier_dfs['lgr']:
    print('\nC:',col)
    p_values_mwu_bymodel(classifier_dfs,col)

### Variance of Distributions unmitigated v mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [ ]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0)
        df = df.iloc[:,1:]
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti)
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
        
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [ ]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')